# TOOL04 : Show SED of Standards GAIA ordered by colors

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/10/31
- last update : 2025/10/31 :
- last update : 2026-01-15 : suppress adjust_Text for usdf

In [ ]:
import sys
sys.executable

In [ ]:
sys.version

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd
import os

In [ ]:
from gaiaspec import getGaia

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Wavelength sampling of.*")

In [ ]:
from getCalspec import getCalspec

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
# where are stored the figures
pathfigs = "figs_TOOL04"
prefix = "tool04"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
def ShowLSSTBands(ax,alpha=0.5,am=1.2,pwv=4.,oz=300.,tau=0.,beta=1.2):
    """
    """
    pc = PhotometricCorrections(am,pwv,oz,tau,beta)

    for index,f in enumerate(filter_tagnames):
        ax.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=alpha) 
        ax.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=alpha) 
        #ax.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4))

ShowLSSTBands(ax,alpha=0.1)

ax.set_xlabel("$\\lambda$ (nm)")
ax.set_title("Total filter throughput (LSST)")

## Cpnnextion to Gaia server

In [ ]:
from astroquery.gaia import Gaia
my_user_name = 'XXXXXXX'
my_password = 'XXXXXXX'
Gaia.login(user=my_user_name, password=my_password)

## Input target file

In [ ]:
listOfGaia = ['HD73495', 'HD104304', 'HD89736', 'HD57167','HD202025', 'HD160760', 'HD36780', 'HD74180', 'HD77020', 'HD111235']

In [ ]:
target_sel ="HD36780"

In [ ]:
gaia = getGaia.Gaia(target_sel)

In [ ]:
gaia.plot_spectrum()

In [ ]:
spec = getGaia.get_gaia_spectra(target_sel)

In [ ]:
assert False

## Show SED grid

In [ ]:
n_stars = len(df)
ncols = 3
nrows = int(np.ceil(n_stars / ncols))
wlmin=300.
wlmax=1200    

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*2.5), sharex=True)
axes = axes.flatten()  # simplifie l’itération


for i, (idx, row) in enumerate(df.iterrows()):
    ax = axes[i]
    
    # Extraire info
    sedname = row["spectr_name"]
    SpT_type = row["Sp_T"]
    color = cmap(norm(type_to_color_idx[SpT_type]))
    
    # Charger la SED avec getCalspec
    c = getCalspec.Calspec(sedname)
    sed = c.get_spectrum_numpy()
    wave, Flambda = sed['WAVELENGTH'].value, sed['FLUX'].value
    wave = wave/10.
    Flambda = Flambda*10.   

    waveindexes = np.where(np.logical_and(wave>wlmin,wave<wlmax))[0]
    wave = wave[waveindexes]
    Flambda = Flambda[waveindexes]
    
    
    # Tracer SED
    label = f"{sedname} ({SpT_type})"
    ax.plot(wave, wave*Flambda, color=color,label=label)  # lambda*Flambda pour signal photonique
    #ax.set_title(f"{name} ({SpT_type})", fontsize=8, color=color)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.legend(loc="upper right")
    ax2 =ax.twinx()
    ShowLSSTBands(ax2,alpha=0.05)

# Supprimer les axes vides
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Axes communs
fig.text(0.5, 0.04, "Wavelength [nm]", ha="center",fontsize=12)
fig.text(0.04, 0.5, r"$\lambda F_\lambda$ [erg/cm2]", va="center", rotation="vertical",fontsize=12)

#plt.suptitle("SED $\lambda F_\lambda$ ",fontsize=16,y=1)
plt.tight_layout(rect=[0.05,0.05,1,0.95])

figname =f"{pathfigs}/{prefix}_allsedlambdaflambda"+figtype
plt.savefig(figname)

plt.show()



In [ ]:
n_stars = len(df)
ncols = 4
nrows = int(np.ceil(n_stars / ncols))
wlmin=300.
wlmax=1200    

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*2.5), sharex=True)
axes = axes.flatten()  # simplifie l’itération


for i, (idx, row) in enumerate(df.iterrows()):
    ax = axes[i]
    
    # Extraire info
    sedname = row["spectr_name"]
    SpT_type = row["Sp_T"]
    color = cmap(norm(type_to_color_idx[SpT_type]))
    
    # Charger la SED avec getCalspec
    c = getCalspec.Calspec(sedname)
    sed = c.get_spectrum_numpy()
    wave, Flambda = sed['WAVELENGTH'].value, sed['FLUX'].value
    wave = wave/10.
    Flambda = Flambda*10.   

    waveindexes = np.where(np.logical_and(wave>wlmin,wave<wlmax))[0]
    wave = wave[waveindexes]
    Flambda = Flambda[waveindexes]
    
    
    # Tracer SED
    label = f"{sedname} ({SpT_type})"
    ax.plot(wave, Flambda, color=color,label=label)  # lambda*Flambda pour signal photonique
    #ax.set_title(f"{name} ({SpT_type})", fontsize=8, color=color)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.legend(loc="upper right")
    ax2 =ax.twinx()
    ShowLSSTBands(ax2,alpha=0.05)

# Supprimer les axes vides
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Axes communs
fig.text(0.5, 0.04, "Wavelength [nm]", ha="center")
fig.text(0.04, 0.5, r"$F_\lambda$ [erg/cm2/nm]", va="center", rotation="vertical")

plt.tight_layout(rect=[0.05,0.05,1,0.95])

figname =f"{pathfigs}/{prefix}_allsedflambda"+figtype
plt.savefig(figname)

plt.show()

